# CSE 5334 Programming Assignment 1 (P1)

In this assignment, you will implement a toy "search engine" in Python. You code will read a corpus and produce TF-IDF vectors for documents in the corpus. Then, given a query string, you code will return the query answer--the document with the highest cosine similarity score for the query. 

The instructions on this assignment are written in an .ipynb file. You can use the following commands to install the Jupyter notebook viewer. You can use the following commands to install the Jupyter notebook viewer. "pip" is a command for installing Python packages. You are required to use Python 3.5.1 or more recent versions of Python in this project. 

    pip install jupyter

    pip install notebook (You might have to use "sudo" if you are installing them at system level)

To run the Jupyter notebook viewer, use the following command:

    jupyter notebook P1.ipynb

The above command will start a webservice at http://localhost:8888/ and display the instructions in the '.ipynb' file.

### Requirements

* This assignment must be done individually. You must implement the whole assignment by yourself. Academic dishonety will have serious consequences.
* You can discuss topics related to the assignment with your fellow students. But you are not allowed to discuss/share your solution and code.

### Dataset

We use a corpus of 40 Inaugural addresses of different US presidents. We processed the corpus and provided you a .zip file, which includes 40 .txt files.

### Programming Language

1. You are required to submit a single .py file of your code.

2. You are expected to use several modules in NLTK--a natural language processing toolkit for Python. NLTK doesn't come with Python by default. You need to install it and "import" it in your .py file. NLTK's website (http://www.nltk.org/index.html) provides a lot of useful information, including a book http://www.nltk.org/book/, as well as installation instructions (http://www.nltk.org/install.html).

3. In programming assignment 1, other than NLTK, you are not allowed to use any other non-standard Python package. However, you are free to use anything from the the Python Standard Library that comes with Python (https://docs.python.org/3/library/).

### Tasks

You code should accomplish the following tasks:

(1) <b>Read</b> the 40 .txt files, each of which has the transcript of inaugural addresses by different US presidents. The following code does it. Make sure to replace "corpusroot" by your directory where the files are stored. In the example below, "corpusroot" is a sub-folder named "US_Inaugural_Addresses" in the folder containing the python file of the code. 

In this assignment we ignore the difference between lower and upper cases. So convert the text to lower case before you do anything else with the text. For a query, also convert it to lower case before you answer the query. 

In [5]:
import os
corpusroot = './US_Inaugural_Addresses'
documents = []
for filename in os.listdir(corpusroot):
    correct_file_name = filename.startswith('0') or filename.startswith('1') or filename.startswith('2') or filename.startswith('3') or filename.startswith('4')
    if correct_file_name:
        file = open(os.path.join(corpusroot, filename), "r", encoding='windows-1252')
        doc = file.read()
        file.close() 
        doc = doc.lower()
        documents.append(doc)

for doc in documents[:5]:
    print(f'doc:{doc}\n')

doc:martin van buren	1837-03-04	fellow-citizens: the practice of all my predecessors imposes on me an obligation i cheerfully fulfill--to accompany the first and solemn act of my public trust with an avowal of the principles that will guide me in performing it and an expression of my feelings on assuming a charge so responsible and vast. in imitating their example i tread in the footsteps of illustrious men, whose superiors it is our happiness to believe are not found on the executive calendar of any country. among them we recognize the earliest and firmest pillars of the republic--those by whom our national independence was first declared, him who above all others contributed to establish it on the field of battle, and those whose expanded intellect and patriotism constructed, improved, and perfected the inestimable institutions under which we live. if such men in the position i now occupy felt themselves overwhelmed by a sense of gratitude for this the highest of all marks of their c

(2) <b>Tokenize</b> the content of each file. For this, you need a tokenizer. For example, the following piece of code uses a regular expression tokenizer to return all course numbers in a string. Play with it and edit it. You can change the regular expression and the string to observe different output results. 

For tokenizing the inaugural Presidential speeches, we will use RegexpTokenizer(r'[a-zA-Z]+')


In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

document_tokens = []
for document in documents:
    document_tokens.append(tokenizer.tokenize(document))

(3) Perform <b>stopword removal</b> on the obtained tokens. NLTK already comes with a stopword list, as a corpus in the "NLTK Data" (http://www.nltk.org/nltk_data/). You need to install this corpus. Follow the instructions at http://www.nltk.org/data.html. You can also find the instruction in this book: http://www.nltk.org/book/ch01.html (Section 1.2 Getting Started with NLTK). Basically, use the following statements in Python interpreter. A pop-up window will appear. Click "Corpora" and choose "stopwords" from the list.

In [12]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


: 

After the stopword list is downloaded, you will find a file "english" in folder nltk_data/corpora/stopwords, where folder nltk_data is the download directory in the step above. The file contains 179 stopwords. nltk.corpus.stopwords will give you this list of stopwords. Try the following piece of code.

In [13]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

print(f"Number of Stop words in english: {len(stop_words)}")

total_words_b = sum(len(doc) for doc in document_tokens)
print(f"Total words before stop word removal is {total_words_b}.")

stop_words_count = 0
for tokens in document_tokens:
    l1 = len(tokens)
    for token in tokens:
        if token in stop_words:
            tokens.remove(token)
            
    stop_words_count += l1 - len(tokens)

total_words_a = sum(len(doc) for doc in document_tokens)
print(f"Total words after stop word removal is {total_words_a}.")

# check if the count matches
if stop_words_count == (total_words_b - total_words_a):
    print("Successfully Removed stopwords.")

Number of Stop words in english: 179
Total words before stop word removal is 47553.
Total words after stop word removal is 47551.
Successfully Removed stopwords.


(4) Also perform <b>stemming</b> on the obtained tokens. NLTK comes with a Porter stemmer. Try the following code and learn how to use the stemmer.

In [14]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

document_tokens_stemmed = []
for document_token in document_tokens:
    stemmed_tokens = []
    for token in document_token:
        stemmed_tokens.append(stemmer.stem(token))
    document_tokens_stemmed.append(stemmed_tokens)

print("Successfully Completed stemming.")

['martin', 'van', 'buren', 'fellow', 'citizen', 'practic', 'predecessor', 'impos', 'oblig', 'cheer', 'fulfil', 'accompani', 'first', 'solemn', 'act', 'public', 'trust', 'avow', 'principl', 'guid', 'perform', 'express', 'feel', 'assum', 'charg', 'respons', 'vast', 'imit', 'exampl', 'tread', 'footstep', 'illustri', 'men', 'whose', 'superior', 'happi', 'believ', 'found', 'execut', 'calendar', 'countri', 'among', 'recogn', 'earliest', 'firmest', 'pillar', 'republ', 'nation', 'independ', 'first', 'declar', 'other', 'contribut', 'establish', 'field', 'battl', 'whose', 'expand', 'intellect', 'patriot', 'construct', 'improv', 'perfect', 'inestim', 'institut', 'live', 'men', 'posit', 'occupi', 'felt', 'overwhelm', 'sens', 'gratitud', 'highest', 'mark', 'countri', 'confid', 'conscious', 'inabl', 'adequ', 'discharg', 'duti', 'offic', 'difficult', 'exalt', 'much', 'must', 'consider', 'affect', 'one', 'reli', 'claim', 'favor', 'forbear', 'unlik', 'preced', 'revolut', 'gave', 'us', 'exist', 'one', '

In [17]:
print(document_tokens_stemmed[0])

['martin', 'van', 'buren', 'fellow', 'citizen', 'the', 'practic', 'of', 'all', 'my', 'predecessor', 'impos', 'on', 'me', 'an', 'oblig', 'i', 'cheer', 'fulfil', 'to', 'accompani', 'the', 'first', 'and', 'solemn', 'act', 'of', 'my', 'public', 'trust', 'with', 'an', 'avow', 'of', 'the', 'principl', 'that', 'will', 'guid', 'me', 'in', 'perform', 'it', 'and', 'an', 'express', 'of', 'my', 'feel', 'on', 'assum', 'a', 'charg', 'so', 'respons', 'and', 'vast', 'in', 'imit', 'their', 'exampl', 'i', 'tread', 'in', 'the', 'footstep', 'of', 'illustri', 'men', 'whose', 'superior', 'it', 'is', 'our', 'happi', 'to', 'believ', 'are', 'not', 'found', 'on', 'the', 'execut', 'calendar', 'of', 'ani', 'countri', 'among', 'them', 'we', 'recogn', 'the', 'earliest', 'and', 'firmest', 'pillar', 'of', 'the', 'republ', 'those', 'by', 'whom', 'our', 'nation', 'independ', 'wa', 'first', 'declar', 'him', 'who', 'abov', 'all', 'other', 'contribut', 'to', 'establish', 'it', 'on', 'the', 'field', 'of', 'battl', 'and', '

In [22]:
# tf-idf
# using the tokens, computer the TF-IDF vector for each document 
    # -> to do that we need 
        # -> term frequency
        # -> document frequency
        # generate count matrix
import pandas as pd

count_matrix = 

for document in document_tokens_stemmed:
    for token in document:
        if token



  (0, 2505)	1
  (0, 4395)	1
  (0, 543)	1
  (0, 1568)	6
  (0, 664)	7
  (0, 4006)	252
  (0, 3032)	2
  (0, 2752)	198
  (0, 139)	30
  (0, 2650)	31
  (0, 3045)	2
  (0, 2047)	3
  (0, 2769)	23
  (0, 2519)	13
  (0, 189)	16
  (0, 2730)	2
  (0, 1997)	39
  (0, 641)	4
  (0, 1728)	1
  (0, 4075)	139
  (0, 37)	1
  (0, 1604)	3
  (0, 194)	150
  (0, 3699)	3
  (0, 55)	1
  :	:
  (39, 3203)	1
  (39, 4166)	5
  (39, 1754)	1
  (39, 963)	1
  (39, 2688)	1
  (39, 711)	1
  (39, 1792)	1
  (39, 3181)	1
  (39, 3016)	2
  (39, 3432)	2
  (39, 4174)	1
  (39, 3886)	1
  (39, 4076)	1
  (39, 3282)	1
  (39, 4036)	2
  (39, 1686)	2
  (39, 3792)	1
  (39, 60)	1
  (39, 2827)	1
  (39, 3651)	1
  (39, 859)	1
  (39, 2410)	2
  (39, 2178)	1
  (39, 500)	2
  (39, 2713)	1


(5) Using the tokens, compute the <b>TF-IDF vector</b> for each document. Use the following equation that we learned in the lectures to calculate the term weights, in which $t$ is a token and $d$ is a document:  $$w_{t,d} = (1+log_{10}{tf_{t,d}})\times(log_{10}{\frac{N}{df_t}}).$$ Note that the TF-IDF vectors should be normalized (i.e., their lengths should be 1). 

Represent a TF-IDF vector by a dictionary. 

(6) Given a query string, calculate the query vector. (Remember to convert it to lower case.) In calculating the query vector, don't consider IDF. I.e., use the following equation to calculate the term weights in the query vector, in which $t$ is a token and $q$ is the query: $$w_{t,q} = (1+log_{10}{tf_{t,q}}).$$
The vector should also be normalized. 

(7) Find the document that attains the highest <b>cosine similarity</b> score. If we compute the cosine similarity between the query vector and every document vector, it is too inefficient. Instead, implement the following method:

(7.1) For each token $t$ that exists in the corpus, construct its <b>postings list</b>---a sorted list in which each element is in the form of (document $d$, TF-IDF weight $w$). Such an element provides $t$'s weight $w$ in document $d$. The elements in the list are sorted by weights in descending order. 

(7.2) For each token $t$ in the query, return the top-10 elements in its corresponding postings list. If the token $t$ doesn't exist in the corpus, ignore it. 

(7.3) If a document $d$ appears in the top-10 elements of every query token, calculate $d$'s cosine similarity score. Recall that the score is defined as follows. Since $d$ appears in top-10 of all query tokens, we have all the information to calculate its actual score $sim(q,d)$.

$$ sim(q,d) = \vec{q} \cdot \vec{d} = \sum_{t\ \text{in both q and d}} w_{t,q} \times w_{t,d}.$$

(7.4) If a document $d$ doesn't appear in the top-10 elements of some query token $t$, use the weight in the 10th element as the upper-bound on $t$'s weight in $d$'s vector. Hence, we can calculate the upper-bound score for $d$ using the query tokens' actual and upper-bound weights with respect to $d$'s vector, as follows. 

$$ \overline{sim(q,d)} = \sum_{t \in T_1} w_{t,q} \times w_{t,d} + \sum_{t\in T_2} w_{t,q} \times \overline{w_{t,d}}.$$

In the above equation, $T_1$ includes query tokens whose top-10 elements contain $d$. $T_2$ includes query tokens whose top-10 elements do not contain $d$. $\overline{w_{t,d}}$ is the weight in the 10-th element of $t$'s postings list. As a special case, for a document $d$ that doesn't appear in the top-10 elements of any query token $t$, its upper-bound score is thus: 

$$ \overline{sim(q,d)} = \sum_{t\in q} w_{t,q} \times \overline{w_{t,d}}.$$

(7.5) If a document's actual score is better than or equal to the actual scores and upper-bound scores of all other documents, it is returned as the query answer. 

If there isn't such a document, it means we need to go deeper than 10 elements into the postings list of each query token. 

### What to Submit 

<b> Submit through Canvas your source code in a single .py file.</b> You can use any standard Python library. The only non-standard library/package allowed for this assignment is NLTK. You .py file must define at least the following functions:

* getidf(token): return the inverse document frequency of a token. If the token doesn't exist in the corpus, return -1. You should stem the parameter 'token' before calculating the idf score.

* getweight(filename,token): return the normalized TF-IDF weight of a token in the document named 'filename'. If the token doesn't exist in the document, return 0. You should stem the parameter 'token' before calculating the tf-idf score.

* query(qstring): return a tuple in the form of (filename of the document, score), where the document is the query answer according to 7.5. If no document contains any token in the query, return ("None",0). If we need more than 10 elements from each posting list, return ("fetch more",0).

In [ ]:
print("%.12f" % getidf('democracy'))
print("%.12f" % getidf('foreign'))
print("%.12f" % getidf('states'))
print("%.12f" % getidf('honor'))
print("%.12f" % getidf('great'))
print("--------------")
print("%.12f" % getweight('19_lincoln_1861.txt','constitution'))
print("%.12f" % getweight('23_hayes_1877.txt','public'))
print("%.12f" % getweight('25_cleveland_1885.txt','citizen'))
print("%.12f" % getweight('09_monroe_1821.txt','revenue'))
print("%.12f" % getweight('37_roosevelt_franklin_1933.txt','leadership'))
print("--------------")
print("(%s, %.12f)" % query("states laws"))
print("(%s, %.12f)" % query("war offenses"))
print("(%s, %.12f)" % query("british war"))
print("(%s, %.12f)" % query("texas government"))
print("(%s, %.12f)" % query("world civilization"))

### Evaluation

Your program will be evaluated using the following criteria: 

* Correctness (75 Points)

We will evaluate your code by calling the functions specificed above (getidf - 20 points; getweight - 25 points; query - 30 points). So, make sure to use the same function names, parameter names/types/orders as specified above. We will use the above test cases and other queries and tokens to test your program.


* Preprocessing, Efficiency, modularity (25 Points)

You should correctly follow the preprocessing steps. An efficient solution should be able to answer a query in a few seconds, you will get deductions if you code takes too long to run (>1 minute). Also, it should consider the boundary cases. Your program should behave correctly under special cases and even incorrect input. Follow good coding standards to make your program easy to understand by others and easy to maintain/extend. 
